# Рекомендация тарифов

Оператор мобильной связи «Мегалайн» выяснил: многие клиенты пользуются архивными тарифами. Они хотят построить систему, способную проанализировать поведение клиентов и предложить пользователям новый тариф: «Смарт» или «Ультра».

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

In [34]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV



# Первичный обзор данных

In [8]:
df = pd.read_csv('users_behavior.csv')


In [9]:
df.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


Набор данных содержит в себе следующие столбцы:

`calls` - количество звонков

`minutes` - суммарная длительность звонков в минутах

`messages` - количество sms-сообщений.

`mb_used` - израсходованный интернет-трафик в Мб.

`is_ultra` - каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0)

Исходя из данных, имеем 5 столбцов и 3214 строк, типы соответствуют действительности и пропуски отсутствуютю

In [11]:
df.describe()

,calls,minutes,messages,mb_used,is_ultra
count,3214.000000,3214.000000,3214.000000,3214.000000,3214.000000
mean,63.038892,438.208787,38.281269,17207.673836,0.306472
std,33.236368,234.569872,36.148326,7570.968246,0.461100
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,274.575000,9.000000,12491.902500,0.000000
50%,62.000000,430.600000,30.000000,16943.235000,0.000000
75%,82.000000,571.927500,57.000000,21424.700000,1.000000
max,244.000000,1632.060000,224.000000,49745.730000,1.000000


По обзору характеристик можно выделить, что среднее количество звонков - 38, минут - 438, сообщений - 38, использованных мегабайт - 17207


**Вывод:**

* Данные не нуждаются в предобработке.
* Пропуски отсутствуют.

# Построение модели

Разобьем данные на три выборки: обучающую, валидационную и тестовую. Разобьем их в отношении 3:1:1. Пока нами не была изучена кросс-валидация, использую для этого метод train_test_split():

In [20]:
df_train, df_valid = train_test_split(df, test_size = 0.4, random_state = 12345)

df_valid, df_test = train_test_split(df_valid, test_size = 0.5, random_state = 12345)

In [21]:
print(df_train.shape, df_valid.shape, df_test.shape)

(1928, 5) (643, 5) (643, 5)


**Вывод:** Выборки разделены.

## Исследуем модели

**Перед исследованием моделей машинного обучения разделим наши датафреймы на целевой признак который нужно предсказать и признаки, которые помогут нам его предсказать:**

In [39]:
features_train = df_train.drop(columns = 'is_ultra')
target_train = df_train['is_ultra']

features_valid = df_valid.drop(columns = 'is_ultra')
target_valid = df_valid['is_ultra']

**Оценим в цикле долю правильных ответов для разных глубин дерева принятия решений:**

In [42]:
for depth in range(2,28,2):
    model = DecisionTreeClassifier(max_depth = depth, random_state = 12345, min_samples_split = 100)
    model.fit(features_train, target_train)
    pred_valid = model.predict(features_valid)
    result = accuracy_score(target_valid, pred_valid)
    print(f'max_depth = {depth} :', '{:.4f}'.format(result))
    

max_depth = 2 : 0.7823
max_depth = 4 : 0.7854
max_depth = 6 : 0.7885
max_depth = 8 : 0.7854
max_depth = 10 : 0.7807
max_depth = 12 : 0.7807
max_depth = 14 : 0.7807
max_depth = 16 : 0.7807
max_depth = 18 : 0.7807
max_depth = 20 : 0.7807
max_depth = 22 : 0.7807
max_depth = 24 : 0.7807
max_depth = 26 : 0.7807



Лучшая модель с глубиной 6 с accuracy = 0.7885. min_samples_split был подобран интуитивно, и улучшил результат на 6 тысячных по сравнению с дефолтным. Любые опыты с критерием и весом классов (balanced) ухудшали результат при min_samples_split = 100.

**Перейдем к алгоритму случайного леса:**

Оценим в цикле долю правильных ответов для разного количества оценщиков

In [29]:
for est in range(1,16):
    model = RandomForestClassifier(n_estimators = est, random_state = 12345, max_depth = 6)
    model.fit(features_train, target_train)
    pred_valid = model.predict(features_valid)
    result = accuracy_score(target_valid, pred_valid)
    print(f'n_estimators = {est} :', '{:.4}'.format(result))

n_estimators = 1 : 0.7854
n_estimators = 2 : 0.7854
n_estimators = 3 : 0.7838
n_estimators = 4 : 0.7885
n_estimators = 5 : 0.7947
n_estimators = 6 : 0.7932
n_estimators = 7 : 0.7947
n_estimators = 8 : 0.7994
n_estimators = 9 : 0.7994
n_estimators = 10 : 0.8009
n_estimators = 11 : 0.8009
n_estimators = 12 : 0.804
n_estimators = 13 : 0.8025
n_estimators = 14 : 0.8025
n_estimators = 15 : 0.8009


Лучшая модель с количеством оценщиков 12 и глубиной 6 с accuracy = 0.8025. min_samples_split лучше оставить дефолтным, он ухудшал результат.

In [43]:
rs.best_params_

{'n_estimators': 91,
 'min_samples_split': 10,
 'max_leaf_nodes': 19,
 'max_features': 3,
 'max_depth': 9,
 'bootstrap': True}

In [44]:
pred = rs.predict(features_valid)
print(accuracy_score(target_valid, pred))

0.7869362363919129


**Перейдем к логистической регрессии:**

In [45]:
model = LogisticRegression(random_state = 12345)
model.fit(features_train, target_train)
pred_valid = model.predict(features_valid)
print(accuracy_score(target_valid, pred_valid))

0.7107309486780715


**Вывод:**

* Мы разделили тренировочную выборку и валидационную на признаки(features) и целевой признак(target).
* Обучили дерево принятия решений и выбрали лучшие с максимальной глубиной 2 и 6.
* Обучили модель случайного леса, и выбрали лучшую с количеством оценщиков 12 и глубиной 6. Ее используем для проверки на тестовой выборке. Также используем модель, для которой подобрали параметры случайным поиском.
* Обучили модель логистической регресии и получили метрики хуже чем у предыдущих моделей.
* Открыли для себя несколько интересных методов, гиперпараметров, потренировались подбирать параметры.

## Проверка модели тестовой выборке

**Перед проверкой модели разделим наш тестовый датафрейм на features и target:**

In [46]:
features_test = df_test.drop(columns = 'is_ultra')
target_test = df_test['is_ultra']

**Проверим модель на тестовой выборке:**

In [47]:
model = RandomForestClassifier(n_estimators = 12, max_depth = 6, random_state = 12345)
model.fit(features_train,target_train)
pred_test = model.predict(features_test)
print(f'accuracy = {accuracy_score(target_test,pred_test)}')

accuracy = 0.7947122861586314


**Проверим модель rs с гиперпараметрами, подобранными случайным поиском на тестовой выборке:**

In [48]:
pred = rs.predict(features_test)
print(accuracy_score(target_test, pred))

0.8009331259720062


Модель, для которой мы подбирали гиперпараметры случайным поиском показала чуть-чуть лучший результат, чем та у которой мы подобрали только глубину и количество оценщиков. Тем не менее обе модели справились с поставленной задачей.

**Вывод:**

* Мы проверили две лучшие подобранные нами модели на тестовой выборке и получили устраивающий нас результат.
* Следующий шаг: научиться пользоваться кросс-валидацией и GridSearch для повышения accuracy в подобных задачах.